# **YOLO-NAS**

Postup vytvorenia modelu YOLO-NAS v tomto notebooku je založený na <a href="https://github.com/roboflow/notebooks/blob/main/notebooks/train-yolo-nas-on-custom-dataset.ipynb">originálnom notebooku.</a>



### Príprava prostredia

In [ ]:
!pip install -q git+https://github.com/Deci-AI/super-gradients.git@stable
!pip install -q supervision
!pip install --upgrade pyparsing cryptography opencv-python-headless pyyaml
!python3 -m pip install pip --upgrade
!pip install pyopenssl --upgrade

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

### Načítanie predtrénovaného modelu

In [ ]:
from super_gradients.training import models
import torch

DEVICE = 'cuda' if torch.cuda.is_available() else "cpu"
MODEL_ARCH = 'yolo_nas_l'

model = models.get(MODEL_ARCH, pretrained_weights="coco").to(DEVICE)

### Nastavenie tried objektov a cesty k datasetu

In [ ]:
LOCATION = "../dataset"
print("location:", LOCATION)
CLASSES = ['TLE']
print("classes:", CLASSES)

### Príprava dát

In [ ]:
BATCH_SIZE = 4
MAX_EPOCHS = 70
CHECKPOINT_DIR = f'{HOME}/checkpoints'
EXPERIMENT_NAME = "TLE_YOLO_NAS_70" # Názov experimentu

In [ ]:
from super_gradients.training import Trainer
trainer = Trainer(experiment_name=EXPERIMENT_NAME, ckpt_root_dir=CHECKPOINT_DIR)

In [ ]:
dataset_params = {
    'data_dir': LOCATION,
    'train_images_dir':'train/images',
    'train_labels_dir':'train/labels',
    'val_images_dir':'val/images',
    'val_labels_dir':'val/labels',
    'test_images_dir':'TP_test/images',
    'test_labels_dir':'TP_test/labels',
    'classes': CLASSES
}

In [ ]:
from super_gradients.training.dataloaders.dataloaders import (
    coco_detection_yolo_format_train, coco_detection_yolo_format_val)

train_data = coco_detection_yolo_format_train(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['train_images_dir'],
        'labels_dir': dataset_params['train_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': BATCH_SIZE,
        'num_workers': 1
    }
)

val_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['val_images_dir'],
        'labels_dir': dataset_params['val_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': BATCH_SIZE,
        'num_workers': 1
    }
)

test_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['test_images_dir'],
        'labels_dir': dataset_params['test_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': BATCH_SIZE,
        'num_workers': 1
    }
)

train_data.dataset.transforms

### Nastavenie konfigurácií pre tréning

In [ ]:
from super_gradients.training import models

model = models.get(
    MODEL_ARCH,
    num_classes=len(dataset_params['classes']),
    pretrained_weights="coco"
)

In [ ]:
from super_gradients.training.losses import PPYoloELoss
from super_gradients.training.metrics import DetectionMetrics_050
from super_gradients.training.models.detection_models.pp_yolo_e import PPYoloEPostPredictionCallback

train_params = {
    'silent_mode': False,
    "average_best_models":True,
    "warmup_mode": "linear_epoch_step",
    "warmup_initial_lr": 1e-6,
    "lr_warmup_epochs": 3,
    "initial_lr": 5e-4,
    "lr_mode": "cosine",
    "cosine_final_lr_ratio": 0.1,
    "optimizer": "Adam",
    "optimizer_params": {"weight_decay": 0.0001},
    "zero_weight_decay_on_bias_and_bn": True,
    "ema": True,
    "ema_params": {"decay": 0.9, "decay_type": "threshold"},
    "max_epochs": MAX_EPOCHS,
    "mixed_precision": True,
    "loss": PPYoloELoss(
        use_static_assigner=False,
        num_classes=len(dataset_params['classes']),
        reg_max=16
    ),
    "valid_metrics_list": [
        DetectionMetrics_050(
            score_thres=0.1,
            top_k_predictions=300,
            num_cls=len(dataset_params['classes']),
            normalize_targets=True,
            post_prediction_callback=PPYoloEPostPredictionCallback(
                score_threshold=0.01,
                nms_top_k=1000,
                max_predictions=300,
                nms_threshold=0.7
            )
        )
    ],
    "metric_to_watch": 'mAP@0.50'
}

### Trénovanie modelu

In [ ]:
trainer.train(
    model=model,
    training_params=train_params,
    train_loader=train_data,
    valid_loader=val_data
)

### Načítanie natrénovaného modelu

In [ ]:
from super_gradients.training import models
best_model = models.get(
    "yolo_nas_l",
    num_classes=len(dataset_params['classes']),
    #checkpoint_path=f"{CHECKPOINT_DIR}/{EXPERIMENT_NAME}/average_model.pth"
    checkpoint_path="checkpoints/TLE_YOLO_NAS_opt/RUN_20240518_064752_800464/average_model.pth"
).to(DEVICE)

### Vizualizácia predikcií

Vykonanie vizualizácie na všetky obrázky v testovacej množine a následné uloženie do súborov:

*inferenced*:
>TP - súbor pre obrázky s detegovanými objektmi
 
>TN - súbor pre obrázky bez detegovaných objektov

In [ ]:
import os
import torch
import cv2
from PIL import Image
import supervision as sv

# Define the directory paths
images_directory_path = "../dataset/test/images"
output_directory_path = "opt_inferenced_ep70_conf04"
tp_directory_path = os.path.join(output_directory_path, "TP")
tn_directory_path = os.path.join(output_directory_path, "TN")

# Create output directories if they don't exist
os.makedirs(tp_directory_path, exist_ok=True)
os.makedirs(tn_directory_path, exist_ok=True)

# Confidence threshold for predictions
CONFIDENCE_THRESHOLD = 0.5

# Dictionary to store predictions for each image
predictions = {}

# Iterate over image files in the directory
for filename in os.listdir(images_directory_path):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        # Read the image
        image_path = os.path.join(images_directory_path, filename)
        image = cv2.imread(image_path)

        # Perform prediction with the model
        result = best_model.predict(image, conf=CONFIDENCE_THRESHOLD, fuse_model=False)

        # Extract predicted bounding boxes, labels, and confidence scores
        bboxes_xyxy = result.prediction.bboxes_xyxy
        labels = result.prediction.labels.astype(int)
        confidence = result.prediction.confidence

        # Store the predictions in a format compatible with supervision
        detections = sv.Detections(xyxy=bboxes_xyxy, confidence=confidence, class_id=labels)

        # Store the predictions for the current image
        predictions[filename] = detections

        # Draw bounding boxes on the image
        for bbox in bboxes_xyxy:
            x1, y1, x2, y2 = bbox
            cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)

        # Determine whether the image contains predicted objects or not
        if len(bboxes_xyxy) > 0:
            output_directory = tp_directory_path
        else:
            output_directory = tn_directory_path

        # Save the image with bounding boxes
        output_image_path = os.path.join(output_directory, filename)
        cv2.imwrite(output_image_path, image)


### Optimalizácia modelu pomocou Evolution Strategy
Search space definuje parametre, ktorými sa optimalizuje model.

In [ ]:
from hyperactive import Hyperactive
from hyperactive.optimizers import EvolutionStrategyOptimizer
from super_gradients.training import Trainer
from super_gradients.training.models.detection_models.pp_yolo_e import PPYoloE
from super_gradients.training.losses import PPYoloELoss
from super_gradients.training.metrics import DetectionMetrics_050
from super_gradients.training.models.detection_models.pp_yolo_e import PPYoloEPostPredictionCallback

def yolo_model(opt):
    MODEL_ARCH = "yolo_nas_l"
    BATCH_SIZE = opt["batch_size"]
    MAX_EPOCHS = opt["epochs"]
                     

    train_data = coco_detection_yolo_format_train(
        dataset_params={
            'data_dir': dataset_params['data_dir'],
            'images_dir': dataset_params['train_images_dir'],
            'labels_dir': dataset_params['train_labels_dir'],
            'classes': dataset_params['classes']
        },
        dataloader_params={
            'batch_size': BATCH_SIZE,
            'num_workers': 1
        }
    )

    val_data = coco_detection_yolo_format_val(
        dataset_params={
            'data_dir': dataset_params['data_dir'],
            'images_dir': dataset_params['val_images_dir'],
            'labels_dir': dataset_params['val_labels_dir'],
            'classes': dataset_params['classes']
        },
        dataloader_params={
            'batch_size': BATCH_SIZE,
            'num_workers': 1
        }
    )

    test_data = coco_detection_yolo_format_val(
        dataset_params={
            'data_dir': dataset_params['data_dir'],
            'images_dir': dataset_params['test_images_dir'],
            'labels_dir': dataset_params['test_labels_dir'],
            'classes': dataset_params['classes']
        },
        dataloader_params={
            'batch_size': BATCH_SIZE,
            'num_workers': 1
        }
    )
                     
    train_data.dataset.transforms
    
    model = models.get(MODEL_ARCH, pretrained_weights="coco").to(DEVICE)
    
    model = models.get(
        MODEL_ARCH,
        num_classes=len(dataset_params['classes']),
        pretrained_weights="coco"
    )

    train_params = {
        'silent_mode': False,
        "average_best_models":True,
        "warmup_mode": "linear_epoch_step",
        "warmup_initial_lr": 1e-6,
        "lr_warmup_epochs": 3,
        "initial_lr": 5e-4,
        "lr_mode": "cosine",
        "cosine_final_lr_ratio": 0.1,
        "optimizer": "Adam",
        "optimizer_params": {"weight_decay": 0.0001},
        "zero_weight_decay_on_bias_and_bn": True,
        "ema": True,
        "ema_params": {"decay": 0.9, "decay_type": "threshold"},
        "max_epochs": MAX_EPOCHS,
        "mixed_precision": True,
        "loss": PPYoloELoss(
        use_static_assigner=False,
        num_classes=len(dataset_params['classes']),
        reg_max=16
        ),
        "valid_metrics_list": [
            DetectionMetrics_050(
                score_thres=0.1,
                top_k_predictions=300,
                num_cls=len(dataset_params['classes']),
                normalize_targets=True,
                post_prediction_callback=PPYoloEPostPredictionCallback(
                    score_threshold=0.01,
                    nms_top_k=1000,
                    max_predictions=300,
                    nms_threshold=0.7
                )
            )
        ],
        "metric_to_watch": 'mAP@0.50'
    }
    
    trainer.train(
        model=model,
        training_params=train_params,
        train_loader=train_data,
        valid_loader=val_data
    )

    # Assuming metric to watch is the best mAP@0.50 score
    best_metric = trainer.best_metric
    return best_metric

# Definícia search space
search_space = {
    "batch_size": [2, 4],
    "epochs": [30, 50, 70],
    "weight_decay": [0.0001, 0.001, 0.1],
}

optimizer = EvolutionStrategyOptimizer(
    mutation_rate=0.5, crossover_rate=0.5, rand_rest_p=0.05
)

hyper = Hyperactive()
hyper.add_search(yolo_model, search_space, optimizer=optimizer, n_iter=10) # Nastavenie počtu iterácií
hyper.run()